Keras model construction/save/load are quite intuitive and clear. 

However, the inference part does not use any information from training, such as changing input layer or use accuracy definition. What if those kind of things are necessary (e.g. when need to fine tune models)? What problems will Keras face？

In [1]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Using TensorFlow backend.


x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 203s - loss: 0.3432 - acc: 0.8948 - val_loss: 0.0845 - val_acc: 0.9748
Epoch 2/12
60000/60000 [==============================] - 203s - loss: 0.1185 - acc: 0.9649 - val_loss: 0.0543 - val_acc: 0.9833
Epoch 3/12
60000/60000 [==============================] - 187s - loss: 0.0906 - acc: 0.9733 - val_loss: 0.0477 - val_acc: 0.9843
Epoch 4/12
60000/60000 [==============================] - 187s - loss: 0.0754 - acc: 0.9772 - val_loss: 0.0393 - val_acc: 0.9871
Epoch 5/12
60000/60000 [==============================] - 186s - loss: 0.0649 - acc: 0.9811 - val_loss: 0.0361 - val_acc: 0.9878
Epoch 6/12
60000/60000 [==============================] - 186s - loss: 0.0582 - acc: 0.9828 - val_loss: 0.0338 - val_acc: 0.9887
Epoch 7/12
60000/60000 [==============================] - 186s - loss: 0.0535 - acc: 0.9841 - val_loss:

In [5]:
model_json = model.to_json()
with open("keras_cnn_model.json", "w") as json_file:
    json_file.write(model_json)
    
model_yaml = model.to_yaml()
with open("keras_cnn_model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

In [4]:
model.save_weights("model.h5")

In [8]:
from keras.models import model_from_json

# load json and create model
json_file = open('keras_cnn_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(x_test, y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model from disk
acc: 99.80%


In [9]:
loaded_model.get_weights

[array([[[[ 0.14241642,  0.24611874,  0.23172449, -0.00923853,  0.10789327,
           -0.12782356, -0.17875621,  0.10897965,  0.24821173, -0.12753782,
            0.25097269,  0.18729126, -0.00984946,  0.00609839, -0.0139327 ,
            0.09369351,  0.08997718, -0.20407329, -0.10771206, -0.32211715,
            0.26489863,  0.02110662,  0.04843066, -0.08853114, -0.17529367,
            0.15858419,  0.1813315 ,  0.12344767,  0.17633697, -0.15790214,
            0.00460623,  0.05709354]],
 
         [[ 0.11337324,  0.07726249,  0.05551154,  0.04398562,  0.17487285,
           -0.24994047,  0.00364594, -0.18701728,  0.13874483,  0.09795892,
            0.02157613,  0.03347043,  0.12899847, -0.10090744,  0.10638479,
            0.15299311,  0.22753596, -0.06729916,  0.03479053, -0.095043  ,
            0.19045164,  0.09679842,  0.10170104,  0.06615455,  0.16017058,
            0.13289459,  0.0242895 ,  0.18349679,  0.00345007, -0.02446037,
            0.18239248, -0.1795353 ]],
 
      